In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.distributions import Normal, kl_divergence

device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')

In [ ]:
from data.dataset import QM9GraphDataset, create_data_loaders
from mgcvae import MGCVAE
from trainer import MGCVAETrainer

In [4]:
dataset = QM9GraphDataset(csv_path='./data/qm9_bbbp.csv')

Processing...


Processing 2142 molecules...
Successfully processed 2142 molecules, failed: 0


Done!


In [5]:
train_loader, val_loader, test_loader = create_data_loaders(dataset, batch_size=4)

Dataset splits — Train: 1713, Val: 214, Test: 215


In [10]:
model = MGCVAE(
    node_dim = 29,
    edge_dim = 4,
    latent_dim = 32,
    max_nodes = 29,
).to(device)